In [5]:
!pip install gym

In [6]:
!pip install swig

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.8 MB/s eta 0:00:00


In [7]:
!pip install gym[box2d]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 47.8 MB/s eta 0:00:00
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp310-cp310-linux_x86_64.whl size=2376136 sha256=b6b603001bc59855fc7ff73f96186f0123bcecdc8ee746c268b4225e17a20cb4
  Stored in directory: /root/.cache/pip/wheels/db/8f/6a/eaaadf056fba10a98d986f6dce954e6201ba3126926fc5ad9e
Successfully built box2d-py
  Attempting uninstall: pygame
    Found existing installation: pygame 2.6.0
    Uninstalling pygame-2.6.0:
      Successfully uninstalled pygame-2.6.0


In [8]:
import gym
import numpy as np

# Initialize the Lunar Lander environment
env = gym.make('LunarLander-v2')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

class QNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

In [10]:
from collections import deque
import random

class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, transition):
        self.buffer.append(transition)

    def sample(self, batch_size):
        transitions = random.sample(self.buffer, batch_size)
        return list(zip(*transitions))

    def __len__(self):
        return len(self.buffer)

In [11]:
import torch.nn.functional as F

# Hyperparameters
BATCH_SIZE = 64
GAMMA = 0.99
EPSILON = 1.0
EPSILON_MIN = 0.01
EPSILON_DECAY = 0.995
TARGET_UPDATE = 10
LEARNING_RATE = 1e-3
REPLAY_BUFFER_CAPACITY = 10000
N_EPISODES = 1000

In [12]:
policy_net = QNetwork(state_dim, action_dim)
target_net = QNetwork(state_dim, action_dim)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
optimizer = optim.Adam(policy_net.parameters(), lr=LEARNING_RATE)
replay_buffer = ReplayBuffer(REPLAY_BUFFER_CAPACITY)

In [42]:
def optimize_model():
    if len(replay_buffer) < BATCH_SIZE:
        return
    transitions = replay_buffer.sample(BATCH_SIZE)

    states = [np.array(t[0]).flatten() for t in transitions]
    # Pad the states to have the same length
    max_length = max(len(s) for s in states)
    states = np.array([np.pad(s, (0, max_length - len(s))) for s in states])
    # Ensure actions are within valid range
    actions = np.array([t[1] if isinstance(t[1], (int, float)) else t[1][0] for t in transitions])
    print("Actions before clipping:", actions) # Print actions before clipping
    actions = np.clip(actions, 0, policy_net(torch.zeros(1, states.shape[1])).shape[1] - 1) # Clip actions to valid range
    print("Actions after clipping:", actions) # Print actions after clipping
     # Rewards are likely scalars
    rewards = np.array([t[2] if isinstance(t[2], (int, float)) else t[2][0] for t in transitions])
    next_states = [np.array(t[3]).flatten() for t in transitions]
    # Pad next states similar to states
    next_states = np.array([np.pad(s, (0, max_length - len(s))) for s in next_states])
    dones = np.array([t[4] if isinstance(t[4], (int, float, bool)) else t[4][0] for t in transitions])


    states = torch.tensor(states, dtype=torch.float32)
    actions = torch.tensor(actions, dtype=torch.int64)
    rewards = torch.tensor(rewards, dtype=torch.float32)
    next_states = torch.tensor(next_states, dtype=torch.float32)
    dones = torch.tensor(dones, dtype=torch.float32)

    state_action_values = policy_net(states).gather(1, actions.unsqueeze(1)).squeeze(1)
    next_state_values = target_net(next_states).max(1)[0]
    expected_state_action_values = rewards + (GAMMA * next_state_values * (1 - dones))

    loss = F.mse_loss(state_action_values, expected_state_action_values)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [43]:
for episode in range(N_EPISODES):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        if np.random.rand() < EPSILON:
            action = env.action_space.sample()
        else:
            with torch.no_grad():
                state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
                action = policy_net(state_tensor).argmax().item()

        next_state, reward, done, _ = env.step(action)

        replay_buffer.push((state, action, reward, next_state, done))
        state = next_state
        total_reward += reward

        optimize_model()

    if EPSILON > EPSILON_MIN:
        EPSILON *= EPSILON_DECAY

    if episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

    print(f"Episode {episode + 1}, Total Reward: {total_reward}")

env.close()

Streaming output truncated to the last 5000 lines.
Actions after clipping: [0.32978973 0.         0.         0.33406478 0.        ]
Actions before clipping: [-0.2413969   0.         -0.65421509 -0.24660783  0.        ]
Actions after clipping: [0. 0. 0. 0. 0.]
Actions before clipping: [ 4.85420242e-05  0.00000000e+00 -1.26148707e+00  5.38825989e-05
  0.00000000e+00]
Actions after clipping: [4.85420242e-05 0.00000000e+00 0.00000000e+00 5.38825989e-05
 0.00000000e+00]
Actions before clipping: [ 0.06427221  0.         -1.30046551  0.06719484  0.        ]
Actions after clipping: [0.06427221 0.         0.         0.06719484 0.        ]
Actions before clipping: [-0.17825079  0.         -0.41791429 -0.18113288  0.        ]
Actions after clipping: [0. 0. 0. 0. 0.]
Actions before clipping: [ 0.32108918  0.         -0.90561259  0.32505503  0.        ]
Actions after clipping: [0.32108918 0.         0.         0.32505503 0.        ]
Actions before clipping: [-0.04404163  0.         -1.18209803 -0.0